In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

In [10]:
# these are the search redshift and equivalent width boundaries
zbin_low = [2.00, 3.00, 4.00, 2.00]
zbin_high = [3.00, 4.00, 5.45, 5.45]

ewbin_low = [0.0, 0.3, 0.6, 1.0, 0.0, 0.3, 0.0]
ewbin_high = [0.3, 0.6, 1.0, 120, 1.0, 1.0, 120]

ewbin_low = [0.3]
ewbin_high = [120.00]
zbin_low = [2.00, 3.00, 4.00]
zbin_high = [3.00, 4.00, 5.45]




In [11]:
mg2_2796 = 2796.3542699	
light = 299792.458
norm_factor = 1.9600132
omega_M = 0.308
omega_L = 1.0 - omega_M
H0 = 67.8/3.08567758e24 # centimeters  
little_h = 100./H0
ro_crit = 1.89000e-29# units are h^2 g cm^(-3)
amu = 1.66053892e-27 # 10^-27; kilograms
mass_mg2 = 24.30506# * amu ;* 1.e3 ; grams



In [12]:
fake3 = pd.read_csv('sigma3_artificial_detections.csv')
real3 = pd.read_csv('sigma3_real_detections.csv')
fake5 = pd.read_csv('sigma5_artificial_detections.csv')
real5 = pd.read_csv('sigma5_real_detections.csv')

In [15]:
real5.loc[(real5.ewlow == 0.3)]

,lowz,highz,ewlow,ewhigh,localN5,dndz,recN5,rec5,rec_dndz,compN5,comp5,comp_dndz
4,2,3.00,0.3,0.6,3,0.750,4.111,0.729,1.027,4.274,0.701,1.068
5,3,4.00,0.3,0.6,3,0.750,4.097,0.732,1.024,4.297,0.698,1.074
6,4,5.45,0.3,0.6,3,0.517,3.944,0.760,0.680,4.435,0.676,0.764
7,2,5.45,0.3,0.6,9,0.652,12.220,0.736,0.885,13.274,0.677,0.961
20,2,3.00,0.3,1.0,3,0.750,4.111,0.729,1.027,4.274,0.701,1.068
21,3,4.00,0.3,1.0,3,0.750,4.097,0.732,1.024,4.297,0.698,1.074
22,4,5.45,0.3,1.0,4,0.689,4.954,0.807,0.854,5.409,0.739,0.932
23,2,5.45,0.3,1.0,10,0.724,13.284,0.752,0.962,14.329,0.697,1.038


In [13]:
###############################################
#
#      5 sigma
#
#

line = "lz  hz  ewlow ewhigh  dz "
line = line + " dx       N  Nrec Ncomp  " 
line = line + "  fN  fNrec fNcomp "
line = line + "fp_disc  fp_rec fp_com"

print(line)
   
real = real5
fake = fake5
for e in range(0, len(ewbin_low)):
    ewlow = ewbin_low[e]
    ewhigh = ewbin_high[e]

    for z in range(0, len(zbin_low)):
        lowz = zbin_low[z]
        highz = zbin_high[z]
        
        # get real subset
        local_real = real.loc[(real.lowz    == lowz ) & 
                              (real.highz   == highz ) & 
                              (real.ewlow  == ewlow ) & 
                              (real.ewhigh == ewhigh )]
        
        # get fake subset
        local_fake = fake.loc[(fake.lowz    == lowz ) & 
                              (fake.highz   == highz ) & 
                              (fake.ewlow  == ewlow ) & 
                              (fake.ewhigh == ewhigh )]

        Xmax = 2.*((omega_M*((1. + highz)**3.) + omega_L)**0.5)/3./omega_M
        Xmin = 2.*((omega_M*((1. + lowz)**3.) + omega_L)**0.5)/3./omega_M
        dxbin_size = Xmax - Xmin
        dxbin_size = np.floor(dxbin_size*10000.0)/10000.0*4
        dzbin_size = highz - lowz
        dzbin_size = np.floor(dzbin_size*10000.0)/10000.0*4


        # get incidence rates
        if  float(local_real.localN5) == 0.0:
            print(lowz, '  ', highz, '     ',
                  ewlow, '  ',ewhigh,'     ------')
            
        if  float(local_real.localN5) > 0.0:
            realN_disc = float(local_real.localN5)
            realN_rec  = float(local_real.recN5)
            realN_com  = float(local_real.compN5)            
            
            realN_disc = np.floor(realN_disc*10000.0)/10000
            realN_rec  = np.floor(realN_rec*10000.0)/10000
            realN_com  = np.floor(realN_com*10000.0)/10000

            fakeN_disc = float(local_fake.localN5)
            fakeN_rec  = float(local_fake.recN5)
            fakeN_com  = float(local_fake.compN5)
            
            fakeN_disc = np.floor(fakeN_disc*10000.0)/10000
            fakeN_rec  = np.floor(fakeN_rec*10000.0)/10000
            fakeN_com  = np.floor(fakeN_com*10000.0)/10000
            
            fp_disc = 1.0 - float(fakeN_disc/realN_disc )
            fp_rec  = 1.0 - float(fakeN_rec/realN_rec ) 
            fp_com  = 1.0 - float(fakeN_com/realN_com )  
            
            fp_disc = np.floor(fp_disc*10000.0)/10000.0
            fp_rec  = np.floor(fp_rec*10000.0)/10000.0
            fp_com  = np.floor(fp_com*10000.0)/10000.0

            dndz      = realN_disc/dzbin_size   
            dndz_rec  = realN_rec/dzbin_size  
            dndz_comp = realN_com/dzbin_size
            
            dndz = np.floor(dndz*10000.0)/10000.0
            dndz_rec  = np.floor(dndz_rec*10000.0)/10000.0
            dndz_comp  = np.floor(dndz_comp*10000.0)/10000.0
            
            
            dndz_fp      = fp_disc*realN_disc/dzbin_size   
            dndz_rec_fp  = fp_rec*realN_rec/dzbin_size  
            dndz_comp_fp = fp_com*realN_com/dzbin_size
            
            dndz_fp = np.floor(dndz_fp*10000.0)/10000.0
            dndz_rec_fp  = np.floor(dndz_rec_fp*10000.0)/10000.0
            dndz_comp_fp  = np.floor(dndz_comp_fp*10000.0)/10000.0
            
            comp = realN_disc/realN_com
            comp = np.floor(comp*10000.0)/10000.0
            print(lowz, highz, '  ',
                  ewlow, ewhigh,'  ',
                  fp_com, comp, realN_disc)
           # print(lowz, highz, '  ',
           #       ewlow, ewhigh,'  ',
           #       dzbin_size ,  dxbin_size, '  ',
           #       realN_disc, realN_rec, realN_com, '  ',
           #       fakeN_disc, fakeN_rec, fakeN_com,'  ',
           #       fp_disc, fp_rec, fp_com)
            
           #  table values


        

lz  hz  ewlow ewhigh  dz  dx       N  Nrec Ncomp    fN  fNrec fNcomp fp_disc  fp_rec fp_com


TypeError: cannot convert the series to <class 'float'>

In [ ]:
#
#      dndz without false positive corrections, 5 sigma
#
#
line = "lz  hz  ewlow ewhigh "
line = line + "  dndz+/-err       dndz_rec+/-err      dndz_comp+/-err " 
print(line)
   
real = real5
fake = fake5
for e in range(0, len(ewbin_low)):
    ewlow = ewbin_low[e]
    ewhigh = ewbin_high[e]
    print()


    for z in range(0, len(zbin_low)):
        lowz = zbin_low[z]
        highz = zbin_high[z]
        
        # get real subset
        local_real = real.loc[(real.lowz    == lowz ) & 
                              (real.highz   == highz ) & 
                              (real.ewlow  == ewlow ) & 
                              (real.ewhigh == ewhigh )]
        
        # get fake subset
        local_fake = fake.loc[(fake.lowz    == lowz ) & 
                              (fake.highz   == highz ) & 
                              (fake.ewlow  == ewlow ) & 
                              (fake.ewhigh == ewhigh )]

        Xmax = 2.*((omega_M*((1. + highz)**3.) + omega_L)**0.5)/3./omega_M
        Xmin = 2.*((omega_M*((1. + lowz)**3.) + omega_L)**0.5)/3./omega_M
        dxbin_size = Xmax - Xmin
        dxbin_size = np.floor(dxbin_size*10000.0)/10000.0*4
        dzbin_size = highz - lowz
        dzbin_size = np.floor(dzbin_size*10000.0)/10000.0*4


        # get incidence rates
        if  float(local_real.localN5) == 0.0:
            print(lowz, '  ', highz, '     ',
                  ewlow, '  ',ewhigh,'     ------')
            
        if  float(local_real.localN5) > 0.0:
            realN_disc = float(local_real.localN5)
            realN_rec  = float(local_real.recN5)
            realN_com  = float(local_real.compN5)            
            
            realN_disc = np.floor(realN_disc*10000.0)/10000
            realN_rec  = np.floor(realN_rec*10000.0)/10000
            realN_com  = np.floor(realN_com*10000.0)/10000

            dndz      = realN_disc/dzbin_size   
            dndz_rec  = realN_rec/dzbin_size  
            dndz_comp = realN_com/dzbin_size
            
            dndz = np.floor(dndz*10000.0)/10000.0
            dndz_rec  = np.floor(dndz_rec*10000.0)/10000.0
            dndz_comp  = np.floor(dndz_comp*10000.0)/10000.0
            
           
            dndz_err = np.floor(dndz/(realN_disc**0.5)*10000.0)/10000.0
            dndz_rec_err  = np.floor(dndz_rec/(realN_disc**0.5)*10000.0)/10000.0
            dndz_comp_err  = np.floor(dndz_comp/(realN_disc**0.5)*10000.0)/10000.0
            
 
            print(lowz, highz, '  ',
                  ewlow, ewhigh,'  ',
                  realN_disc, dndz, dndz_err, '   ',
                  realN_rec, dndz_rec, dndz_rec_err, '   ', 
                  realN_com, dndz_comp,dndz_comp_err)
        

In [ ]:
#
#      dndz with false positive corrections, 5 sigma
#
#
line = "lz  hz  ewlow ewhigh "
line = line + "    fdndz+/-err       fdndz_rec+/-err       fdndz_comp+/-err " 
print(line)
real = real5
fake = fake5
for e in range(0, len(ewbin_low)):
    ewlow = ewbin_low[e]
    ewhigh = ewbin_high[e]
    print()
    for z in range(0, len(zbin_low)):
        lowz = zbin_low[z]
        highz = zbin_high[z]
        
        # get real subset
        local_real = real.loc[(real.lowz    == lowz ) & 
                              (real.highz   == highz ) & 
                              (real.ewlow  == ewlow ) & 
                              (real.ewhigh == ewhigh )]
        
        # get fake subset
        local_fake = fake.loc[(fake.lowz    == lowz ) & 
                              (fake.highz   == highz ) & 
                              (fake.ewlow  == ewlow ) & 
                              (fake.ewhigh == ewhigh )]

        Xmax = 2.*((omega_M*((1. + highz)**3.) + omega_L)**0.5)/3./omega_M
        Xmin = 2.*((omega_M*((1. + lowz)**3.) + omega_L)**0.5)/3./omega_M
        dxbin_size = Xmax - Xmin
        dxbin_size = np.floor(dxbin_size*10000.0)/10000.0*4
        dzbin_size = highz - lowz
        dzbin_size = np.floor(dzbin_size*10000.0)/10000.0*4


        # get incidence rates
        if  float(local_real.localN5) == 0.0:
            print(lowz, '  ', highz, '     ',
                  ewlow, '  ',ewhigh,'     ------')
            
        if  float(local_real.localN5) > 0.0:
            realN_disc = float(local_real.localN5)
            realN_rec  = float(local_real.recN5)
            realN_com  = float(local_real.compN5)            
            
            realN_disc = np.floor(realN_disc*10000.0)/10000
            realN_rec  = np.floor(realN_rec*10000.0)/10000
            realN_com  = np.floor(realN_com*10000.0)/10000

            fakeN_disc = float(local_fake.localN5)
            fakeN_rec  = float(local_fake.recN5)
            fakeN_com  = float(local_fake.compN5)
              
            fakeN_disc = np.floor(fakeN_disc*10000.0)/10000
            fakeN_rec  = np.floor(fakeN_rec*10000.0)/10000
            fakeN_com  = np.floor(fakeN_com*10000.0)/10000
            
            
            fp_disc = 1.0 - float(fakeN_disc/realN_disc )
            fp_rec  = 1.0 - float(fakeN_rec/realN_rec ) 
            fp_com  = 1.0 - float(fakeN_com/realN_com )  
            
            fp_disc = np.floor(fp_disc*10000.0)/10000.0
            fp_rec  = np.floor(fp_rec*10000.0)/10000.0
            fp_com  = np.floor(fp_com*10000.0)/10000.0


            dndz_fp      = fp_disc*realN_disc/dzbin_size   
            dndz_rec_fp  = fp_rec*realN_rec/dzbin_size  
            dndz_comp_fp = fp_com*realN_com/dzbin_size

            #print(lowz, highz, '  ',
            #      ewlow, ' ', ewhigh,' ', realN_disc/dzbin_size, dndz_fp)
            
            dndz_fp = np.floor(dndz_fp*10000.0)/10000.0
            dndz_rec_fp  = np.floor(dndz_rec_fp*10000.0)/10000.0
            dndz_comp_fp  = np.floor(dndz_comp_fp*10000.0)/10000.0
            
            dndz_fp_err = dndz_fp/(realN_disc**0.5)
            dndz_rec_fp_err = dndz_rec_fp/(realN_rec**0.5)
            dndz_comp_fp_err = dndz_comp_fp/(realN_com**0.5)
 
            dndz_fp_err = np.floor(dndz_fp_err*10000.0)/10000.0
            dndz_rec_fp_err = np.floor(dndz_rec_fp_err*10000.0)/10000.0
            dndz_comp_fp_err = np.floor(dndz_comp_fp_err*10000.0)/10000.0
            
            print(lowz, highz, '  ',
                  ewlow, ' ', ewhigh,'    ',
                  realN_disc, dndz_fp, dndz_fp_err, '   ',
                  realN_rec, dndz_rec_fp, dndz_rec_fp_err, '   ',
                  realN_com, dndz_comp_fp, dndz_comp_fp_err)
        

In [ ]:
#
#      dndx without false positive corrections, 5 sigma
#
#
line = "lz  hz  ewlow ewhigh "
line = line + "  dndx+/-err       dndx_rec+/-err      dndx_comp+/-err " 
print(line)

real = real5
fake = fake5
for e in range(0, len(ewbin_low)):
    ewlow = ewbin_low[e]
    ewhigh = ewbin_high[e]
    print()


    for z in range(0, len(zbin_low)):
        lowz = zbin_low[z]
        highz = zbin_high[z]
        
        # get real subset
        local_real = real.loc[(real.lowz    == lowz ) & 
                              (real.highz   == highz ) & 
                              (real.ewlow  == ewlow ) & 
                              (real.ewhigh == ewhigh )]
        
        # get fake subset
        local_fake = fake.loc[(fake.lowz    == lowz ) & 
                              (fake.highz   == highz ) & 
                              (fake.ewlow  == ewlow ) & 
                              (fake.ewhigh == ewhigh )]

        Xmax = 2.*((omega_M*((1. + highz)**3.) + omega_L)**0.5)/3./omega_M
        Xmin = 2.*((omega_M*((1. + lowz)**3.) + omega_L)**0.5)/3./omega_M
        dxbin_size = Xmax - Xmin
        dxbin_size = np.floor(dxbin_size*10000.0)/10000.0*4
        dzbin_size = highz - lowz
        dzbin_size = np.floor(dzbin_size*10000.0)/10000.0*4


        # get incidence rates
        if  float(local_real.localN5) == 0.0:
            print(lowz, '  ', highz, '     ',
                  ewlow, '  ',ewhigh,'     ------')
            
        if  float(local_real.localN5) > 0.0:
            realN_disc = float(local_real.localN5)
            realN_rec  = float(local_real.recN5)
            realN_com  = float(local_real.compN5)            
            
            realN_disc = np.floor(realN_disc*10000.0)/10000
            realN_rec  = np.floor(realN_rec*10000.0)/10000
            realN_com  = np.floor(realN_com*10000.0)/10000

            fakeN_disc = float(local_fake.localN5)
            fakeN_rec  = float(local_fake.recN5)
            fakeN_com  = float(local_fake.compN5)
            
            
            dndx      = realN_disc/dxbin_size   
            dndx_rec  = realN_rec/dxbin_size  
            dndx_comp = realN_com/dxbin_size
            
            dndx = np.floor(dndx*10000.0)/10000.0
            dndx_rec  = np.floor(dndx_rec*10000.0)/10000.0
            dndx_comp  = np.floor(dndx_comp*10000.0)/10000.0
            
            dndx_err = np.floor(dndx/(realN_disc**0.5)*10000.0)/10000.0
            dndx_rec_err  = np.floor(dndx_rec/(realN_disc**0.5)*10000.0)/10000.0
            dndx_comp_err  = np.floor(dndx_comp/(realN_disc**0.5)*10000.0)/10000.0
            
 
            print(lowz, highz, '  ',
                  ewlow, ewhigh,'  ',
                  realN_disc, dndx, dndx_err, '   ',
                  realN_rec, dndx_rec, dndx_rec_err, '   ', 
                  realN_com, dndx_comp,dndx_comp_err)
          
      

In [ ]:
#
#      dndx with false positive corrections, 5 sigma
#
#
line = "lz  hz  ewlow ewhigh "
line = line + "  dndx+/-err       dndx_rec+/-err      dndx_comp+/-err " 
print(line)



real = real5
fake = fake5
for e in range(0, len(ewbin_low)):
    ewlow = ewbin_low[e]
    ewhigh = ewbin_high[e]
    print()


    for z in range(0, len(zbin_low)):
        lowz = zbin_low[z]
        highz = zbin_high[z]
        
        # get real subset
        local_real = real.loc[(real.lowz    == lowz ) & 
                              (real.highz   == highz ) & 
                              (real.ewlow  == ewlow ) & 
                              (real.ewhigh == ewhigh )]
        
        # get fake subset
        local_fake = fake.loc[(fake.lowz    == lowz ) & 
                              (fake.highz   == highz ) & 
                              (fake.ewlow  == ewlow ) & 
                              (fake.ewhigh == ewhigh )]

        Xmax = 2.*((omega_M*((1. + highz)**3.) + omega_L)**0.5)/3./omega_M
        Xmin = 2.*((omega_M*((1. + lowz)**3.) + omega_L)**0.5)/3./omega_M
        dxbin_size = Xmax - Xmin
        dxbin_size = np.floor(dxbin_size*10000.0)/10000.0*4
        dzbin_size = highz - lowz
        dzbin_size = np.floor(dzbin_size*10000.0)/10000.0*4


        # get incidence rates
        if  float(local_real.localN5) == 0.0:
            print(lowz, '  ', highz, '     ',
                  ewlow, '  ',ewhigh,'     ------')
            
        if  float(local_real.localN5) > 0.0:
            realN_disc = float(local_real.localN5)
            realN_rec  = float(local_real.recN5)
            realN_com  = float(local_real.compN5)            
            
            realN_disc = np.floor(realN_disc*10000.0)/10000
            realN_rec  = np.floor(realN_rec*10000.0)/10000
            realN_com  = np.floor(realN_com*10000.0)/10000

            fakeN_disc = float(local_fake.localN5)
            fakeN_rec  = float(local_fake.recN5)
            fakeN_com  = float(local_fake.compN5)
            
            fakeN_disc = np.floor(fakeN_disc*10000.0)/10000
            fakeN_rec  = np.floor(fakeN_rec*10000.0)/10000
            fakeN_com  = np.floor(fakeN_com*10000.0)/10000
            
            
            fp_disc = 1.0 - float(fakeN_disc/realN_disc )
            fp_rec  = 1.0 - float(fakeN_rec/realN_rec ) 
            fp_com  = 1.0 - float(fakeN_com/realN_com )  
            
            fp_disc = np.floor(fp_disc*10000.0)/10000.0
            fp_rec  = np.floor(fp_rec*10000.0)/10000.0
            fp_com  = np.floor(fp_com*10000.0)/10000.0



            dndx      = fp_disc*realN_disc/dxbin_size   
            dndx_rec  = fp_rec*realN_rec/dxbin_size  
            dndx_comp = fp_com*realN_com/dxbin_size
            
            dndx = np.floor(dndx*10000.0)/10000.0
            dndx_rec  = np.floor(dndx_rec*10000.0)/10000.0
            dndx_comp  = np.floor(dndx_comp*10000.0)/10000.0
            
            dndx_err = np.floor(dndx/(realN_disc**0.5)*10000.0)/10000.0
            dndx_rec_err  = np.floor(dndx_rec/(realN_disc**0.5)*10000.0)/10000.0
            dndx_comp_err  = np.floor(dndx_comp/(realN_disc**0.5)*10000.0)/10000.0
            
 
            print(lowz, highz, '  ',
                  ewlow, ewhigh,'  ',
                  realN_disc, dndx, dndx_err, '   ',
                  realN_rec, dndx_rec, dndx_rec_err, '   ', 
                  realN_com, dndx_comp,dndx_comp_err)
        
            